In [1]:
import pandas as pd
import json
import hdbscan
from collections import defaultdict

# === 1. Cargar autores desde lab_nodes.csv ===
lab_nodes_df = pd.read_csv("../data/networks/lab_nodes.csv")
author_names = lab_nodes_df['Label'].tolist()

# === 2. Cargar el JSON con los datos completos ===
with open("../../data/data/IPBLN/json/gesbib_authors_ipbln_info.json", encoding="utf-8") as f:
    data_json = json.load(f)

# === 3. Crear un diccionario de nombre -> palabrasClave ===
# Hay que buscar el nombre en las firmas o en firmaDigitalCsic
author_keyword_weights = defaultdict(dict)

for id_autor, autor_data in data_json.items():
    # Todas las firmas posibles
    posibles_nombres = set()
    if 'firmaDigitalCsic' in autor_data:
        posibles_nombres.add(autor_data['firmaDigitalCsic'].strip())
    if 'firmas' in autor_data:
        posibles_nombres.update(f.strip() for f in autor_data['firmas'])

    # Buscar coincidencia con los autores de lab_nodes
    for nombre in author_names:
        if nombre in posibles_nombres:
            palabras = autor_data.get('palabrasClave', {})
            for kw, peso in palabras.items():
                author_keyword_weights[nombre][kw] = peso

# === 4. Crear DataFrame con autores como filas y keywords como columnas ===
df = pd.DataFrame.from_dict(author_keyword_weights, orient='index').fillna(0).astype(int)

# === 5. Opcional: guardar a CSV ===
df.to_csv("Keywords/author_keyword_matrix.csv")

# Mostrar resumen
print("✅ Dataset generado con forma:", df.shape)
print("🔢 Palabras clave totales:", len(df.columns))
print("👥 Autores con keywords:", len(df))


✅ Dataset generado con forma: (29, 3329)
🔢 Palabras clave totales: 3329
👥 Autores con keywords: 29
